[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roedersen/keras-OpenSlideGenerator/blob/master/train_camyleon_resnet.ipynb)

In [ ]:
!pip install tripy

In [ ]:
!pip install pyclipper

In [ ]:
!apt-get install openslide-tools

In [ ]:
!pip install openslide-python

In [ ]:
!git clone https://github.com/roedersen/keras-OpenSlideGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
labels = '/content/drive/MyDrive/Dokumente/FourthBrain/capstone/data/labels/'
images = '/tmp/'

In [ ]:
import sys
sys.path.append("/content/keras-OpenSlideGenerator")
import openslide_generator

In [ ]:
!cp /content/drive/MyDrive/CAMELYON17/training/center_0/patient_000.zip /tmp
!unzip /tmp/patient_000.zip -d /tmp/center_0/

In [ ]:
!cp /content/drive/MyDrive/CAMELYON17/training/center_1/patient_038.zip /tmp
!unzip /tmp/patient_038.zip -d /tmp/center_1/

In [ ]:
!cp /content/drive/MyDrive/CAMELYON17/training/center_2/patient_044.zip /tmp
!unzip /tmp/patient_044.zip -d /tmp/center_2/

In [ ]:
!cp /content/drive/MyDrive/CAMELYON17/training/center_3/patient_074.zip /tmp
!unzip /tmp/patient_074.zip -d /tmp/center_3/

In [ ]:
!cp /content/drive/MyDrive/CAMELYON17/training/center_4/patient_080.zip /tmp
!unzip /tmp/patient_080.zip -d /tmp/center_4/

In [ ]:
from matplotlib import cm
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import RMSprop
import os
import math
import time
import numpy as np
from datetime import datetime
import openslide_generator

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, ReLU
from tensorflow.keras.layers import BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.applications.xception import Xception

In [ ]:
def plot_loss_history(training_history, logscale=False):
    loss = training_history['loss']
    val_loss = training_history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, color='red', label='Training loss')
    plt.plot(epochs, val_loss, color='green', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    if logscale:
        plt.yscale('log')
    plt.show()

In [ ]:
def plot_accuracy_history(training_history):
    acc = training_history['accuracy']
    val_acc = training_history['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, color='red', label='Training acc')
    plt.plot(epochs, val_acc, color='green', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
pixel = 227

In [ ]:
gen_train = openslide_generator.OpenSlideGenerator(labels + 'labels_20220506_train.txt', \
                                                   images, 512, pixel, \
                                                   fetch_mode='label-slide', \
                                                   label_to_use=0)
print("gen_train defined")
print(f'Shape of gen_train: {gen_train.shape()}')

In [ ]:

gen_val = openslide_generator.OpenSlideGenerator(labels + 'labels_20220425_val.txt', \
                                                 images, 512, pixel, \
                                                 fetch_mode='label-slide', \
                                                 label_to_use=0)
print("gen_val defined")

In [ ]:
batch_size = 3
num_classes = len(gen_train.labels[gen_train.label_to_use])
epochs = 1
data_augmentation = False
num_predictions = 20
t_steps = math.ceil(gen_train.patch_per_epoch/batch_size)
v_steps = math.ceil(gen_val.patch_per_epoch/batch_size)

In [ ]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [ ]:
def ResNet34(input_shape = x_pred.shape, num_classes = num_classes):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(num_classes, activation = 'sigmoid')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [ ]:
resnet = ResNet34(input_shape=gen_train.shape(), num_classes=num_classes)

resnet.compile(optimizer=Adam(learning_rate=1.0e-06),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
training_resnet34 = resnet.fit(gen_train.flow(batch_size=batch_size), \
    steps_per_epoch=t_steps, \
    validation_data=gen_val.flow(batch_size=batch_size), \
    validation_steps=v_steps, \
    workers=16, \
    use_multiprocessing=True, \
    verbose=1)

In [ ]:
now = datetime.now()
resnet.save('/content/drive/MyDrive/Dokumente/FourthBrain/capstone/model/resnet_' + str(now.strftime("%Y%m%d_%H%M")),save_format='tf')
print('Saved trained model')

In [ ]:
plot_accuracy_history(training_resnet34.history)
plot_loss_history(training_resnet34.history)
plot_loss_history(training_resnet34.history, logscale=True)